In [145]:
import importlib
import predict
from predict import CoalFirePredictor, predictor, predict_from_csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
importlib.reload(predict)


feature_cols = ["Марка", "Возраст_дн", "mass", "Максимальная температура", 
               "Темп_изменение", "weekday", "month", "t", "p", "humidity"]

Модель загружена: <class 'sklearn.ensemble._forest.RandomForestClassifier'>
LabelEncoder загружен.


# 1. Тест на ручных данных

In [146]:
# тест на ручных данных - разные сценарии
manual_data = pd.DataFrame({
    'Марка': ['A1', 'E5', 'A1', 'A1', 'E5'],
    'Возраст_дн': [30, 150, 250, 45, 180], 
    'mass': [1000, 3000, 6000, 1500, 4500], 
    'Максимальная температура': [25, 60, 85, 35, 75], 
    'Темп_изменение': [1, 5, 12, -2, 8], 
    'weekday': [1, 3, 5, 2, 4],
    'month': [6, 7, 8, 9, 10], 
    't': [20, 30, 35, 25, 15],  
    'p': [1013, 1008, 1005, 1015, 1020], 
    'humidity': [70, 45, 30, 60, 80] 
})

manual_result = predictor.add_predictions_to_df(manual_data, feature_cols)
manual_result.to_csv('../test_analysis/manual_test.csv', index=False)

print("=== ТЕСТ НА РУЧНЫХ ДАННЫХ ===")
print(manual_result[['Марка', 'Возраст_дн', 'Максимальная температура', 'fire_proba', 'fire_pred']])
print(f"\nВысокий риск: {manual_result['fire_pred'].sum()} из {len(manual_result)}")
print(f"Максимальная вероятность: {manual_result['fire_proba'].max():.3f}")

=== ТЕСТ НА РУЧНЫХ ДАННЫХ ===
  Марка  Возраст_дн  Максимальная температура  fire_proba  fire_pred
0    A1          30                        25    0.015997          0
1    E5         150                        60    0.343958          1
2    A1         250                        85    0.471101          1
3    A1          45                        35    0.003461          0
4    E5         180                        75    0.129095          1

Высокий риск: 3 из 5
Максимальная вероятность: 0.471


# 2. Анализ 2019 года

In [147]:
# Загружаем данные 2019
supplies_df = pd.read_csv('../data/supplies/supplies.csv')
temp_df = pd.read_csv('../data/temperature/temperature.csv')
weather_2019_df = pd.read_csv('../data/weather_data/weather_data_2019.csv')
fires_df = pd.read_csv('../data/fires/fires.csv')

# Реальные пожары 2019
fires_2019 = fires_df.copy()
fires_2019['Дата начала'] = pd.to_datetime(fires_2019['Дата начала'], errors='coerce')
fires_2019 = fires_2019[fires_2019['Дата начала'].dt.year == 2019]

print(f"Реальных пожаров в 2019: {len(fires_2019)}")
print(f"Штабели с пожарами: {sorted(fires_2019['Штабель'].unique())}")

Реальных пожаров в 2019: 180
Штабели с пожарами: [np.int64(1), np.int64(4), np.int64(6), np.int64(9), np.int64(10), np.int64(13), np.int64(15), np.int64(16), np.int64(18), np.int64(21), np.int64(22), np.int64(40), np.int64(42), np.int64(46), np.int64(48)]


In [148]:
# Создаем прогнозы на 2019 год
unique_stacks = supplies_df['Штабель'].unique()[:25]  # 25 штабелей
schedule_2019 = []

for stack in unique_stacks:
    stack_info = supplies_df[supplies_df['Штабель'] == stack].iloc[0]
    
    # Каждый месяц по 2 записи
    for month in range(1, 13):
        for day in [15, 30]:
            if month == 2 and day == 30:
                day = 28
            
            current_date = pd.to_datetime(f'2019-{month:02d}-{day:02d}')
            
            # Температура штабеля
            temp_data = temp_df[(temp_df['Штабель'] == stack)]
            max_temp = temp_data['Максимальная температура'].max() if not temp_data.empty else 30
            max_temp = max(25, max_temp + np.random.uniform(-5, 15))
            
            # Погода
            weather_data = weather_2019_df[pd.to_datetime(weather_2019_df['date']) == current_date]
            if not weather_data.empty:
                weather_row = weather_data.iloc[0]
                temp_air, pressure, humidity = weather_row['t'], weather_row['p'], weather_row['humidity']
            else:
                temp_air, pressure, humidity = 15, 1013, 70
            
            schedule_2019.append({
                'Штабель': stack, 'Дата': current_date, 'Марка': stack_info['Наим. ЕТСНГ'],
                'Возраст_дн': (current_date.dayofyear - 1) + np.random.uniform(30, 80),
                'mass': 1500 + np.random.uniform(500, 3000),
                'Максимальная температура': max_temp,
                'Темп_изменение': np.random.uniform(-2, 6),
                'weekday': current_date.weekday(), 'month': current_date.month,
                't': temp_air, 'p': pressure, 'humidity': humidity
            })

schedule_2019_df = pd.DataFrame(schedule_2019)
result_2019 = predictor.add_predictions_to_df(schedule_2019_df, feature_cols)
result_2019.to_csv('../test_analysis/predictions_2019.csv', index=False)

print(f"Прогнозы 2019: {len(result_2019)} записей, {result_2019['fire_pred'].sum()} предсказанных пожаров")

Прогнозы 2019: 600 записей, 148 предсказанных пожаров


In [ ]:
# Анализ и визуализация 2019 года
result_2019['Дата'] = pd.to_datetime(result_2019['Дата'])
monthly_pred_2019 = result_2019.groupby(result_2019['Дата'].dt.month)['fire_pred'].sum()
monthly_fires_2019 = fires_2019.groupby(fires_2019['Дата начала'].dt.month).size()

# Создаем визуализацию
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
months = range(1, 13)
x = np.arange(len(months))

# График 1: Сравнение предсказаний и реальных пожаров
ax1 = axes[0, 0]
pred_2019 = [monthly_pred_2019.get(m, 0) for m in months]
real_2019 = [monthly_fires_2019.get(m, 0) for m in months]
ax1.bar(x - 0.2, pred_2019, 0.4, label='Предсказания', alpha=0.8, color='blue')
ax1.bar(x + 0.2, real_2019, 0.4, label='Реальные пожары', alpha=0.8, color='red')
ax1.set_title('2019: Предсказания vs Реальные пожары')
ax1.set_xlabel('Месяц')
ax1.set_ylabel('Количество пожаров')
ax1.set_xticks(x)
ax1.set_xticklabels(months)
ax1.legend()
ax1.grid(True, alpha=0.3)

# График 2: Вероятности по месяцам
ax2 = axes[0, 1]
monthly_proba_2019 = result_2019.groupby(result_2019['Дата'].dt.month)['fire_proba'].mean()
prob_2019 = [monthly_proba_2019.get(m, 0) for m in months]
ax2.plot(months, prob_2019, 'g-o', linewidth=2, markersize=6)
ax2.set_title('Средняя вероятность пожара по месяцам 2019')
ax2.set_xlabel('Месяц')
ax2.set_ylabel('Вероятность')
ax2.grid(True, alpha=0.3)

# График 3: Распределение вероятностей
ax3 = axes[1, 0]
ax3.hist(result_2019['fire_proba'], bins=30, alpha=0.7, color='purple', edgecolor='black')
# ax3.axvline(0.2, color='red', linestyle='--', label='Порог (0.2)')
ax3.set_title('Распределение вероятностей пожара 2019')
ax3.set_xlabel('Вероятность')
ax3.set_ylabel('Частота')
ax3.legend()
ax3.grid(True, alpha=0.3)

# График 4: Статистика по штабелям
ax4 = axes[1, 1]
stack_pred = result_2019.groupby('Штабель')['fire_pred'].sum().sort_values(ascending=False).head(10)
ax4.bar(range(len(stack_pred)), stack_pred.values, alpha=0.8, color='orange')
ax4.set_title('Топ 10 штабелей по предсказанным пожарам')
ax4.set_xlabel('Штабели (топ 10)')
ax4.set_ylabel('Количество предсказанных пожаров')
ax4.set_xticks(range(len(stack_pred)))
ax4.set_xticklabels([f'Ш{s}' for s in stack_pred.index], rotation=45)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../test_analysis/analysis_2019.png', dpi=300, bbox_inches='tight')
plt.show()

# Детальная статистика
accuracy_2019 = min(sum(pred_2019), sum(real_2019)) / max(sum(pred_2019), sum(real_2019)) * 100

print(f"\n=== АНАЛИЗ 2019 ГОДА ===")
print(f"Предсказано пожаров: {sum(pred_2019)}")
print(f"Реальных пожаров: {sum(real_2019)}")
print(f"Точность модели: {accuracy_2019:.1f}%")
print(f"Максимальная вероятность: {result_2019['fire_proba'].max():.3f}")
print(f"Средняя вероятность: {result_2019['fire_proba'].mean():.3f}")

print(f"\nТоп месяцы по предсказаниям:")
top_pred = monthly_pred_2019.sort_values(ascending=False).head(3)
for month, count in top_pred.items():
    print(f"  Месяц {month}: {count} предсказанных пожаров")

print(f"\nТоп месяцы по реальным пожарам:")
top_real = monthly_fires_2019.sort_values(ascending=False).head(3)
for month, count in top_real.items():
    print(f"  Месяц {month}: {count} реальных пожаров")

print(f"\nШтабели с реальными пожарами в 2019: {sorted(fires_2019['Штабель'].unique())}")
print(f"Штабели с предсказанными пожарами: {sorted(result_2019[result_2019['fire_pred']==1]['Штабель'].unique())}")

# Пересечение штабелей
real_stacks = set(fires_2019['Штабель'].unique())
pred_stacks = set(result_2019[result_2019['fire_pred']==1]['Штабель'].unique())
intersection = real_stacks.intersection(pred_stacks)
print(f"Совпадения штабелей: {sorted(intersection)} ({len(intersection)} из {len(real_stacks)})")
print(f"Точность по штабелям: {len(intersection)/len(real_stacks)*100:.1f}%")


=== АНАЛИЗ 2019 ГОДА ===
Предсказано пожаров: 148
Реальных пожаров: 180
Точность модели: 82.2%
Максимальная вероятность: 0.590
Средняя вероятность: 0.095

Топ месяцы по предсказаниям:
  Месяц 7: 28 предсказанных пожаров
  Месяц 6: 27 предсказанных пожаров
  Месяц 8: 27 предсказанных пожаров

Топ месяцы по реальным пожарам:
  Месяц 9: 42 реальных пожаров
  Месяц 8: 41 реальных пожаров
  Месяц 11: 40 реальных пожаров

Штабели с реальными пожарами в 2019: [np.int64(1), np.int64(4), np.int64(6), np.int64(9), np.int64(10), np.int64(13), np.int64(15), np.int64(16), np.int64(18), np.int64(21), np.int64(22), np.int64(40), np.int64(42), np.int64(46), np.int64(48)]
Штабели с предсказанными пожарами: [np.int64(1), np.int64(5), np.int64(6), np.int64(9), np.int64(12), np.int64(19), np.int64(24), np.int64(30), np.int64(33), np.int64(34), np.int64(35), np.int64(40), np.int64(41), np.int64(44), np.int64(46), np.int64(48), np.int64(50)]
Совпадения штабелей: [np.int64(1), np.int64(6), np.int64(9), np.i

C:\Users\HP\AppData\Local\Temp\ipykernel_13276\684952261.py:58: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


# 3. Анализ 2020 года

In [150]:
# Загружаем данные 2020
weather_2020_df = pd.read_csv('../data/weather_data/weather_data_2020.csv')

# Реальные пожары 2020
fires_2020 = fires_df.copy()
fires_2020['Дата начала'] = pd.to_datetime(fires_2020['Дата начала'], errors='coerce')
fires_2020 = fires_2020[fires_2020['Дата начала'].dt.year == 2020]

print(f"Реальных пожаров в 2020: {len(fires_2020)}")
print(f"Штабели с пожарами: {sorted(fires_2020['Штабель'].unique())}")

Реальных пожаров в 2020: 305
Штабели с пожарами: [np.int64(1), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(12), np.int64(23), np.int64(33), np.int64(38), np.int64(39), np.int64(41)]


In [151]:
# Создаем прогнозы на 2020 год
unique_stacks = supplies_df['Штабель'].unique()[:25]  # 25 штабелей
schedule_2020 = []

for stack in unique_stacks:
    stack_info = supplies_df[supplies_df['Штабель'] == stack].iloc[0]
    
    # Каждый месяц по 2 записи
    for month in range(1, 13):
        for day in [15, 30]:
            if month == 2 and day == 30:
                day = 28
            
            current_date = pd.to_datetime(f'2020-{month:02d}-{day:02d}')
            
            # Температура штабеля
            temp_data = temp_df[(temp_df['Штабель'] == stack)]
            max_temp = temp_data['Максимальная температура'].max() if not temp_data.empty else 30
            max_temp = max(25, max_temp + np.random.uniform(-5, 15))
            
            # Погода
            weather_data = weather_2020_df[pd.to_datetime(weather_2020_df['date']) == current_date]
            if not weather_data.empty:
                weather_row = weather_data.iloc[0]
                temp_air, pressure, humidity = weather_row['t'], weather_row['p'], weather_row['humidity']
            else:
                temp_air, pressure, humidity = 15, 1013, 70
            
            schedule_2020.append({
                'Штабель': stack, 'Дата': current_date, 'Марка': stack_info['Наим. ЕТСНГ'],
                'Возраст_дн': (current_date.dayofyear - 1) + np.random.uniform(30, 80),
                'mass': 1500 + np.random.uniform(500, 3000),
                'Максимальная температура': max_temp,
                'Темп_изменение': np.random.uniform(-2, 6),
                'weekday': current_date.weekday(), 'month': current_date.month,
                't': temp_air, 'p': pressure, 'humidity': humidity
            })

schedule_2020_df = pd.DataFrame(schedule_2020)

In [152]:
result_2020 = predictor.add_predictions_to_df(schedule_2020_df, feature_cols)
result_2020.to_csv('../test_analysis/predictions_2020.csv', index=False)

print(f"Прогнозы 2020: {len(result_2020)} записей, {result_2020['fire_pred'].sum()} предсказанных пожаров")

Прогнозы 2020: 600 записей, 135 предсказанных пожаров


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Группируем по месяцам
result_2020['Дата'] = pd.to_datetime(result_2020['Дата'])
monthly_pred_2020 = result_2020.groupby(result_2020['Дата'].dt.month)['fire_pred'].sum()
monthly_fires_2020 = fires_2020.groupby(fires_2020['Дата начала'].dt.month).size()

# Создаем визуализацию
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
months = range(1, 13)
x = np.arange(len(months))

# График 1: Предсказания vs Реальные пожары
ax1 = axes[0, 0]
pred_2020 = [monthly_pred_2020.get(m, 0) for m in months]
real_2020 = [monthly_fires_2020.get(m, 0) for m in months]
ax1.bar(x - 0.2, pred_2020, 0.4, label='Предсказания', alpha=0.8, color='blue')
ax1.bar(x + 0.2, real_2020, 0.4, label='Реальные пожары', alpha=0.8, color='red')
ax1.set_title('2020: Предсказания vs Реальные пожары')
ax1.set_xlabel('Месяц')
ax1.set_ylabel('Количество пожаров')
ax1.set_xticks(x)
ax1.set_xticklabels(months)
ax1.legend()
ax1.grid(True, alpha=0.3)

# График 2: Средняя вероятность по месяцам
ax2 = axes[0, 1]
monthly_proba_2020 = result_2020.groupby(result_2020['Дата'].dt.month)['fire_proba'].mean()
prob_2020 = [monthly_proba_2020.get(m, 0) for m in months]
ax2.plot(months, prob_2020, 'g-o', linewidth=2, markersize=6)
ax2.set_title('Средняя вероятность пожара по месяцам 2020')
ax2.set_xlabel('Месяц')
ax2.set_ylabel('Вероятность')
ax2.grid(True, alpha=0.3)

# График 3: Распределение вероятностей
ax3 = axes[1, 0]
ax3.hist(result_2020['fire_proba'], bins=30, alpha=0.7, color='purple', edgecolor='black')
# ax3.axvline(0.2, color='red', linestyle='--', label='Порог (0.2)')
ax3.set_title('Распределение вероятностей пожара 2020')
ax3.set_xlabel('Вероятность')
ax3.set_ylabel('Частота')
ax3.legend()
ax3.grid(True, alpha=0.3)

# График 4: Топ 10 штабелей по предсказанным пожарам
ax4 = axes[1, 1]
stack_pred = result_2020.groupby('Штабель')['fire_pred'].sum().sort_values(ascending=False).head(10)
ax4.bar(range(len(stack_pred)), stack_pred.values, alpha=0.8, color='orange')
ax4.set_title('Топ 10 штабелей по предсказанным пожарам 2020')
ax4.set_xlabel('Штабели (топ 10)')
ax4.set_ylabel('Количество предсказанных пожаров')
ax4.set_xticks(range(len(stack_pred)))
ax4.set_xticklabels([f'Ш{s}' for s in stack_pred.index], rotation=45)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../test_analysis/analysis_2020.png', dpi=300, bbox_inches='tight')
plt.show()

# Детальная статистика
accuracy_2020 = min(sum(pred_2020), sum(real_2020)) / max(sum(pred_2020), sum(real_2020)) * 100

print(f"\n=== АНАЛИЗ 2020 ГОДА ===")
print(f"Предсказано пожаров: {sum(pred_2020)}")
print(f"Реальных пожаров: {sum(real_2020)}")
print(f"Точность модели: {accuracy_2020:.1f}%")
print(f"Максимальная вероятность: {result_2020['fire_proba'].max():.3f}")
print(f"Средняя вероятность: {result_2020['fire_proba'].mean():.3f}")

print(f"\nТоп месяцы по предсказаниям:")
top_pred = monthly_pred_2020.sort_values(ascending=False).head(3)
for month, count in top_pred.items():
    print(f"  Месяц {month}: {count} предсказанных пожаров")

print(f"\nТоп месяцы по реальным пожарам:")
top_real = monthly_fires_2020.sort_values(ascending=False).head(3)
for month, count in top_real.items():
    print(f"  Месяц {month}: {count} реальных пожаров")

print(f"\nШтабели с реальными пожарами в 2020: {sorted(fires_2020['Штабель'].unique())}")
print(f"Штабели с предсказанными пожарами: {sorted(result_2020[result_2020['fire_pred']==1]['Штабель'].unique())}")

# Пересечение штабелей
real_stacks = set(fires_2020['Штабель'].unique())
pred_stacks = set(result_2020[result_2020['fire_pred']==1]['Штабель'].unique())
intersection = real_stacks.intersection(pred_stacks)
print(f"Совпадения штабелей: {sorted(intersection)} ({len(intersection)} из {len(real_stacks)})")
print(f"Точность по штабелям: {len(intersection)/len(real_stacks)*100:.1f}%")



=== АНАЛИЗ 2020 ГОДА ===
Предсказано пожаров: 135
Реальных пожаров: 305
Точность модели: 44.3%
Максимальная вероятность: 0.499
Средняя вероятность: 0.091

Топ месяцы по предсказаниям:
  Месяц 9: 28 предсказанных пожаров
  Месяц 7: 27 предсказанных пожаров
  Месяц 6: 25 предсказанных пожаров

Топ месяцы по реальным пожарам:
  Месяц 7: 103 реальных пожаров
  Месяц 9: 70 реальных пожаров
  Месяц 6: 67 реальных пожаров

Штабели с реальными пожарами в 2020: [np.int64(1), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(12), np.int64(23), np.int64(33), np.int64(38), np.int64(39), np.int64(41)]
Штабели с предсказанными пожарами: [np.int64(1), np.int64(5), np.int64(6), np.int64(9), np.int64(12), np.int64(19), np.int64(24), np.int64(30), np.int64(33), np.int64(34), np.int64(35), np.int64(40), np.int64(41), np.int64(44), np.int64(46), np.int64(48), np.int64(50)]
Совпадения штабелей: [np.int64(1), np.int64(5), np.int64(6), np.int64(12), np.int64(33), np.int64(41)] (6 из 11)
Точность 

C:\Users\HP\AppData\Local\Temp\ipykernel_13276\2034969340.py:62: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
